In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import time, re
import pandas as pd

- 2라운드 진출 식당 46곳 스크롤 내리면서 페이지 들락날락 하는 코드입니다. 필요에 맞게 변형해서 사용하시면 됩니다!

In [17]:
# 웹드라이버 설정 및 페이지 열기
driver = webdriver.Chrome()
driver.get("https://app.catchtable.co.kr/ct/curation/culinaryclasswars?tabIndex=0&currentExhibitionKey=classwars-all&isUseExhibitionFilter=1&hasShopRefsFromClient=1&curationQuickFilterKey=2&serviceType=INTEGRATION&uniqueListId=1728841646438&metaContractedType=0&sortMethod=recommended&centerBoundsLat=35.60447951909298&centerBoundsLng=127.62260840243572&zoomLevel=13&isNewSearchInMap=1")

In [13]:
# 코드 단축 함수
def getElement(link) :
    return driver.find_element(By.CLASS_NAME, link)
def getElements(link) :
    return driver.find_elements(By.CLASS_NAME, link)

# 현재 페이지의 식당 정보 크롤링 함수
def get_res_info(target_text) :
    
    #해당 페이지에 방문한 식당 데이터 추가
    name_chef = getElement('j8dkby0').text.split("\n")[0].split("로 출연한 ")[-1]
    try :
        nick_name = getElement('j8dkby0').text.split("\n")[0].split("로 출연한 ")[0].strip("으")
    except :
        nick_name = ""
    
    name_restaurant = target_text
    
    #현재 식당링크 :
    link_now = driver.current_url

    #평점 정보
    try : 
        rating = float(getElement('dl6idsb').text)
        rating_cnt = getElement('dl6idsc').text
        rating_cnt = int("".join(re.findall(r'\d+', rating_cnt)))
    except :
        rating = 0.0
        rating_cnt = 0
        print("평점 정보 X")

    #식당 설명
    style_restaurant = getElement('dl6ids4').text.split("\n")[-1]
    try :
        desc_restaurant = getElement('mxtve20').text
    except :
        desc_restaurant = ""
        print("식당 설명 X")

    #특이 사항
    etc_restaurant = ""
    etc_restaurant += getElements('_1ltqxco1g')[3].text.replace("\n"," , ") + ""
    etc_restaurant += " , ".join(getElements('v-scroll-inner')[1].text.split())

    #영업 시간 -> DB에 어떤 형태로 들어가는지에 따라 보완 예정
    #getElements('_1ltqxco1g')[-1].click()
    #time_restaurant = 

    #식당 위치
    getElement('_1ltqxco1g').click()
    loca_res = getElement('zn9ch54').text.split("\n")[0]
    
    tmp = [name_chef, nick_name , name_restaurant
           ,loca_res, link_now
            ,rating, rating_cnt
           , style_restaurant, desc_restaurant
            ,etc_restaurant
           ]
    return tmp

In [14]:
# 페이지 로딩 대기
time.sleep(2)

visited_restaurants = []  # 방문한 식당 텍스트 저장
restaurant_info = [] # 방문한 식당정보 텍스트 저장

cnt = 0


def scroll_and_find_restaurant(target_text):
    """스크롤을 내리며 특정 식당을 찾아 클릭"""
    prev_height = 0
    while True:
        # 현재 페이지의 모든 식당 요소 가져오기
        elements = driver.find_elements(By.CLASS_NAME, "ShopListItem_title__1p45wh65.app_typo_headline_h2__jg3o1w1a")

        # 식당을 찾아 클릭
        for element in elements:
            if element.text == target_text:
                element.click()  # 해당 식당 클릭
                time.sleep(1)  # 페이지 로딩 대기
                #print(f"visited: {target_text}")
                
                restaurant_info.append(get_res_info(target_text))
                
                driver.back()  # 뒤로 가기
                time.sleep(1)  # 페이지 로딩 대기
                return  # 함수 종료



def find_and_visit_new_restaurants():
    """스크롤하며 새로운 식당을 찾고 방문"""
    prev_height = 0

    while True:
        elements = driver.find_elements(By.CLASS_NAME, "ShopListItem_title__1p45wh65.app_typo_headline_h2__jg3o1w1a")

        new_restaurants = []
        for element in elements:
            text = element.text
            if text not in visited_restaurants:  # 중복 방지
                print(f"New restaurant found: {text}")
                visited_restaurants.append(text)
                new_restaurants.append(text)

        # 새로 발견한 식당 방문
        for restaurant in new_restaurants:
            scroll_and_find_restaurant(restaurant)

        # 스크롤 다운
        down_height = prev_height + 1500 # 아래로 1500만큼 스크롤
        driver.execute_script(f"window.scrollTo(0, {down_height});")
        time.sleep(1)  # 스크롤 후 페이지 로딩 대기

        # 스크롤 종료 조건 확인
        new_height = driver.execute_script("return window.scrollY")
        if new_height == prev_height:
            break  # 더 이상 스크롤되지 않으면 종료
        prev_height = new_height


# 메인 실행
find_and_visit_new_restaurants()


# 브라우저 종료
driver.quit()

New restaurant found: 식당네오
New restaurant found: 로컬릿x스텔라 고메데이
New restaurant found: 리북방x스텔라 고메데이
New restaurant found: 비아톨레도 파스타바
New restaurant found: 디핀 deepin
New restaurant found: 트리드 (trid)
New restaurant found: 디핀 옥수
New restaurant found: CHOI. 쵸이닷
평점 정보 X
식당 설명 X
평점 정보 X
식당 설명 X
New restaurant found: 도량
New restaurant found: 티엔미미 홍대
New restaurant found: PONO BUONO 포노 부오노
New restaurant found: 조광201
New restaurant found: 프렌치 터치더스카이
New restaurant found: 캐비스트리
New restaurant found: 달리아 다이닝
New restaurant found: 네기스키야키 압구정점
New restaurant found: 부토(BUTO)
New restaurant found: 네기 라이브
New restaurant found: 본연
New restaurant found: 마마리마켓&펍
New restaurant found: 진진
New restaurant found: 중앙감속기
New restaurant found: 모던오뎅 신사점
New restaurant found: 윤서울
New restaurant found: 군몽
New restaurant found: 로컬릿
New restaurant found: 홍보각
New restaurant found: 중앙가속기
New restaurant found: 가매일식
평점 정보 X
New restaurant found: 남영탉
New restaurant found: 을지로보석
New restaurant found: 야키토리묵
New restaurant fou

In [15]:
cols = ["셰프","닉네임", "식당","주소","URL","별점","리뷰 수","스타일", "매장 설명", "기타 정보"]
df = pd.DataFrame(restaurant_info, columns = cols)

In [16]:
df

,셰프,닉네임,식당,주소,URL,별점,리뷰 수,스타일,매장 설명,기타 정보
0,최강록,최강록,식당네오,서울특별시 송파구 삼전로12길 4 1층 101호,https://app.catchtable.co.kr/ct/shop/neo_?type...,4.9,642,일식,마스터 셰프 코리아2 우승자 최강록 셰프의 술안주 위주의 요리를 코스로 제공하는 주점,"점심 영업안함 , 저녁 7.7만원룸 , 노키즈존 , 단체 , 이용가능 , 홀 , 바"
1,남정석,남정석,로컬릿x스텔라 고메데이,서울특별시 성동구 한림말길 33 청훈빌딩,https://app.catchtable.co.kr/ct/shop/localeats...,0.0,0,이탈리아음식,,"점심 영업안함 , 디너 6만원 (2인 코스)홀"
2,최지형,최지형,리북방x스텔라 고메데이,서울특별시 마포구 마포대로1길 16 A&D빌딩,https://app.catchtable.co.kr/ct/shop/leebukban...,0.0,0,한식,,"점심 영업안함 , 디너 7.5만원홀"
3,권성준,나폴리 맛피아,비아톨레도 파스타바,서울특별시 용산구 원효로83길 7-2 1층,https://app.catchtable.co.kr/ct/shop/viatoledo...,4.8,494,파스타,나폴리를 담고 있는 파스타바,"점심 영업안함 , 저녁 가격 변동콜키지 , 가능 , 단체 , 이용가능 , 대관 , 가능"
4,윤남노,요리하는 돌아이,디핀 deepin,서울특별시 중구 퇴계로 411 1층,https://app.catchtable.co.kr/ct/shop/deepin?ty...,4.9,1038,다이닝바,흑백요리사 요리하는 돌아이 윤남노 셰프의 음식과 분위기가 완벽한 와인바,"점심 영업안함 , 저녁 1-5만원콜키지 , 가능 , 노키즈존 , 단체 , 이용가능 ..."
5,강승원,트리플스타,트리드 (trid),서울특별시 강남구 선릉로162길 16 2층,https://app.catchtable.co.kr/ct/shop/trid?type...,4.8,667,코스요리,미쉐린 3스타 레스토랑 출신 흑백요리사 트리플스타 강승원 셰프가 풀어내는 다국적 감...,"점심 12만원 , 저녁 20만원주차 , 가능 , 발렛 , 가능 , 콜키지 , 가능 ..."
6,윤남노,요리하는 돌아이,디핀 옥수,서울특별시 성동구 독서당로 194 지하 1층,https://app.catchtable.co.kr/ct/shop/deepin_ok...,4.6,2031,파스타,흑백요리사 요리하는 돌아이 윤남노 셰프의 아시안 터치를 가미한 Fresh Pasta...,"점심, 저녁 동일가 , 2-3만원콜키지 , 가능 , 노키즈존 , 단체 , 이용가능 ..."
7,최현석,최현석,CHOI. 쵸이닷,서울특별시 강남구 도산대로 457 앙스돔빌딩 3층,https://app.catchtable.co.kr/ct/shop/choidot?t...,4.5,2066,이탈리아음식,대한민국 대표 스타셰프 최현석 셰프의 독창적인 요리를 즐길 수 있는 이탈리안 기반의...,"점심 9.8만원 , 저녁 19.8만원주차 , 가능 , 발렛 , 가능 , 콜키지 , ..."
8,임태훈,철가방 요리사,도량,"서울특별시 종로구 자하문로6길 6 백송빌딩 2, 3층",https://app.catchtable.co.kr/ct/shop/dolyang?t...,4.8,119,중식,맛있는 훠궈와 중화요리가 있는 식당,"양고기 튀김 , 30,000원대기공간 , 주차 , 가능 , 콜키지 , 가능 , 단체..."
9,정지선,정지선,티엔미미 홍대,서울특별시 마포구 양화로 144 MERCURE 앰배서더 HOTEL 홍대 4층,https://app.catchtable.co.kr/ct/shop/tianmimih...,4.5,85,중식,한국 최초 여성 중식 스타 셰프 정지선 셰프의 첫 만남의 기억처럼 계속 생각나는 딤...,"티엔미미 해산물냉채 , 41,000원단체 , 이용가능 , 무선인터넷"


In [11]:
df.to_csv("식당정보.csv")